target format is JSON
{
"demoset":{"bottle":"pullo","station":"asema","shelf":"hylly","bar":"baari"},
"demoset2":{"bottle2":"pullo2","station2":"asema2","shelf2":"hylly2","bar2":"baari2"}
 }

In [160]:
import pandas as pd

In [262]:
nouns = pd.read_excel("./datasets/core words completed.xlsx", 0 )
verbs = pd.read_excel("./datasets/core words completed.xlsx", 2)
adj = pd.read_excel("./datasets/core words completed.xlsx", 1 )
rankings = pd.read_csv("./datasets/rankedvocab.csv")

In [263]:
nouns["POS1"] = "n"
nouns.head()
nouns.shape

(2459, 3)

In [264]:
verbs.head()
verbs.columns = ["Finnish","English"]
verbs["POS1"] = "v"
verbs.shape

(1138, 3)

In [265]:
adj.head()
adj["POS1"] = "j"
adj.shape

(569, 3)

In [270]:
rankings.head()
rankings["POS"].value_counts()

POS
n       2688
v       1042
j        692
r        214
fw       167
u         99
m         84
K         62
abbr      14
Name: count, dtype: int64

In [282]:
rankings[rankings["POS"]=="r"].shape

(214, 8)

In [274]:
rankingreduced = rankings[["word","POS","ranking"]]
# rankingreduced = rankings[["word","ranking"]]

rankingreduced.head()
rankingreduced.shape

(5063, 3)

In [310]:
stage1 = pd.merge(rankingreduced, adj, left_on = "word", right_on= "English", how = "right", left_index = False, right_index = False)
stage1.shape

(653, 6)

In [286]:
stage1.head()

,word,POS,ranking,Finnish,English,POS1
0,able,j,609.0,kykenevä,able,j
1,above,fw,1555.0,ylinen,above,j
2,absolute,j,3511.0,absoluuttinen,absolute,j
3,absurd,j,4740.0,absurdi,absurd,j
4,acceptable,j,4477.0,hyväksyttävä,acceptable,j


In [312]:
stage1[stage1["POS1"]=="j"].shape

(653, 6)

In [320]:
stage1["Finnish"].isna().sum()

0

In [324]:
s1clean = stage1[(stage1["Finnish"].notnull()) &((stage1["POS1"]=="j") | (stage1["POS1"]=="r" )|(stage1["POS1"]=="fw"))].copy()
# s1clean = stage1[(stage1["Finnish"].notnull())].copy()

# s1clean["POS"] = "rj" #adjs
# # stage1[(stage1["Finnish"].notnull())&(stage1["POS"]=="j")].shape #adjs
s1clean.shape

(653, 6)

In [326]:
s1clean.head()

,word,POS,ranking,Finnish,English,POS1
0,able,j,609.0,kykenevä,able,j
1,above,fw,1555.0,ylinen,above,j
2,absolute,j,3511.0,absoluuttinen,absolute,j
3,absurd,j,4740.0,absurdi,absurd,j
4,acceptable,j,4477.0,hyväksyttävä,acceptable,j


In [330]:
stage2 = pd.merge(rankingreduced, nouns, left_on = "word", right_on= "English", how = "right", left_index = False, right_index = False)

In [332]:
s2clean = stage2[(stage2["Finnish"].notnull()) & (stage2["POS1"]=="n") ] #nouns
s2clean.shape

(2824, 6)

In [334]:
stage3 = pd.merge(rankingreduced, verbs, left_on = "word", right_on= "English", how = "right", left_index = False, right_index = False)

In [336]:
s3clean = stage3[stage3["Finnish"].notnull()]
s3clean.shape
#verbs

(1438, 6)

In [338]:
#adj + v + n
569+1138+2459


4166

In [59]:
total = pd.concat([s1clean,s2clean,s3clean], ignore_index = True)

In [113]:
total[(total["ranking"]>= 0) & (total["ranking"]<5000)]["POS"].value_counts()

POS
n     2499
v     1201
rj     492
j      157
r       26
u       16
m       14
fw      10
Name: count, dtype: int64

In [91]:
# now split by ranking 
total[ ((total["ranking"]>= 0) & (total["ranking"]<500)) & (total['POS']=="v")].shape

(165, 5)

In [93]:
total[ ((total["ranking"]>= 0) & (total["ranking"]<500)) & (total['POS']=="n")].shape

(139, 5)

In [105]:
total[ ((total["ranking"]>= 0) & (total["ranking"]<500)) & (total['POS']=="fw")].shape

(2, 5)

In [103]:
bounds = [0,500,1000,1500,2000,2500,3000,3500,4000]
lower = bounds[0]
upper = bounds[1]

POS = ["v","n","rj"]

batch = [ total[((total["ranking"]>= lower) & (total["ranking"]<upper)) & (total['POS']== pos)] for pos in POS]
pd.concat(batch).shape
name = f"{lower}-{higher}"
   
# def setbuilder(lower,upper):
#     [total[ ((total["ranking"]> lower) & (total["ranking"]<upper)) & (total['POS']== pos)].shape for pos in POS

(352, 5)

In [197]:
total.to_csv("./check me.csv", encoding = "utf-8-sig")

In [1]:
help(pd.read_excel)

NameError: name 'pd' is not defined

In [23]:
# help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'MergeHow' = 'inner', on: 'IndexLabel | AnyArrayLike | None' = None, left_on: 'IndexLabel | AnyArrayLike | None' = None, right_on: 'IndexLabel | AnyArrayLike | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool | None' = None, indicator: 'str | bool' = False, validate: 'str | None' = None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.

    A named Series object is treated as a DataFrame with a single named column.

    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.

  

In [169]:
#Manual translate required 
u = rankings[rankings["POS"]=="u"]
fw = rankings[rankings["POS"]=="fw"]
abbr = rankings[rankings["POS"]=="abbr"]
r = rankings[rankings["POS"]=="r"]

pd.concat([fw,u,abbr,r], ignore_index = True)

,index,Summary,word,POS,freq,forms,important,ranking
0,2,I|fw,I,fw,267578.0,I,vital,2
1,3,you|fw,you,fw,248266.0,you,vital,3
2,4,the|fw,the,fw,176446.0,the,vital,4
3,5,a|fw,a,fw,152598.0,"a, an",vital,5
4,6,to|fw,to,fw,151270.0,to,vital,6
...,...,...,...,...,...,...,...,...
489,4770,horribly|r,horribly,r,36.0,horribly,bonus,4770
490,4902,poorly|r,poorly,r,34.0,poorly,bonus,4902
491,5043,sincerely|r,sincerely,r,32.0,sincerely,bonus,5043
492,5008,elsewhere|r,elsewhere,r,32.0,elsewhere,bonus,5008
